In [4]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tkinter as tk
from PIL import Image, ImageTk
import time
from collections import deque
from tensorflow import keras

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

class ClassifierApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Epilepsy Monitoring")
        self.root.minsize(900, 700)
        self.root.configure(bg="aqua")
        self.model_to_select = "None"
        
        self.canvas = tk.Canvas(root, bg="aqua", highlightthickness=0)
        self.canvas.pack(side="left", fill="both", expand=True)

        self.scrollbar = tk.Scrollbar(root, command=self.canvas.yview)
        self.scrollbar.pack(side="right", fill="y")

        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.frame = tk.Frame(self.canvas, bg="aqua")
        self.canvas.create_window((0, 0), window=self.frame, anchor="nw")

        self.start_Button = tk.Button(self.frame, text="START", command=self.call_model)
        self.start_Button.grid(row=5, column=0, padx=300, sticky='w', pady=(10,100), columnspan=3)

        self.is_classifying = False
        self.frame_count = 0

        self.frame.bind("<Configure>", self.on_configure)
        self.root.bind("<MouseWheel>", self._on_mousewheel)

    def on_configure(self, event):
        self.canvas.configure(scrollregion=self.canvas.bbox('all'))

    def _on_mousewheel(self, event):
        self.canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

    def call_model(self):
        model_path = "C:/models"
        my_model = keras.models.load_model(model_path, compile=False)
        mp_holistic = mp.solutions.holistic
        frame_size = 75 #3 secs frame

        face_landmark_indices = [0, 4, 17, 48, 50, 61, 122, 130, 133, 145, 159, 206, 280, 289, 292, 351, 362, 359, 374, 386, 426]

        start_time = None
        time_elapsed = 0
        timer_running = False
        with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
            cap = cv2.VideoCapture(0)
            frame_buffer = []
            while cap.isOpened():
                frame_landmarks = []
                ret, frame = cap.read()
                frame = cv2.flip(frame, 1)
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                try:
                    if results.pose_landmarks:
                        for landmark in results.pose_landmarks.landmark:
                            frame_landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
                            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                                      landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                    else:
                        frame_landmarks.extend([0.0] * (33 * 4))

                    if results.face_landmarks:
                        for i in face_landmark_indices:
                            landmark = results.face_landmarks.landmark[i]
                            frame_landmarks.extend([landmark.x, landmark.y, landmark.z])
                            mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                                      landmark_drawing_spec=None,
                                                      connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
                    else:
                        frame_landmarks.extend([0.0] * (len(face_landmark_indices) * 3))

                    if results.left_hand_landmarks:
                        for landmark in results.left_hand_landmarks.landmark:
                            frame_landmarks.extend([landmark.x, landmark.y, landmark.z])
                            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                                      mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                                      mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))
                    else:
                        frame_landmarks.extend([0.0] * (21 * 3))

                    if results.right_hand_landmarks:
                        for landmark in results.right_hand_landmarks.landmark:
                            frame_landmarks.extend([landmark.x, landmark.y, landmark.z])
                            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                                      mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                                      mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
                    else:
                        frame_landmarks.extend([0.0] * (21 * 3))
                except Exception as e:
                    print("An error occurred:", e)
                frame_landmarks = np.array(frame_landmarks)

                if len(frame_landmarks) == 321:
                    frame_buffer.append(frame_landmarks.copy())

                if len(frame_buffer) >= frame_size:
                    input_sequence = np.array(frame_buffer).reshape(1, frame_size, 321)
                    result = my_model(input_sequence)
                    print("The result is ", result)
                    result_index = np.argmax(result)
                    if result_index != 0:
                        activity_name = "Seizure"
                        if not timer_running:
                            start_time = time.time() - time_elapsed
                            timer_running = True
                    else:
                        activity_name = "Normal"
                        timer_running = False

                    if timer_running:
                        time_elapsed = time.time() - start_time
                    cv2.putText(image, f"Duration : {int(time_elapsed)}s", (20, 80), cv2.FONT_HERSHEY_TRIPLEX, 0.8, (180, 0, 100), 2, bottomLeftOrigin=False)
                    cv2.putText(image, str(activity_name), (20, 50), cv2.FONT_HERSHEY_TRIPLEX, 0.8, (180, 0, 100), 2, bottomLeftOrigin=False)

                    cv2.imshow('Pose Classification', image)
                    frame_buffer = frame_buffer[1:]

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    root = tk.Tk()
    app = ClassifierApp(root)
    root.mainloop()


The result is  tf.Tensor([[0.59062666 0.46119702]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.5313556 0.5138188]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.47565803 0.5704044 ]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.4360365  0.60984963]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.52102697 0.5243442 ]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.70787674 0.38522616]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.89388466 0.19957179]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.8942792 0.1997057]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.8984367  0.19416545]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.8994413  0.19286598]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.8983464  0.19447353]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.8946225  0.19961464]], shape=(1, 2), dtype=float32)
The result is  tf.Tensor([[0.881